In [1]:
import pandas as pd
import io
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
#Importing the data in chunks; increases time but decreases memory consumption
def main():
    
    #Creates a list with dictionaries
    for run in range(0,12):
        cols = np.arange(6,20688)+(20682*run)
        indexes = np.arange(6,20688,2)+(20682*run)
        allels = pd.read_csv('filtered_conv.ped',sep = '\t',header=None, usecols=cols)
        
        #Create a directory containing information on each allele
        for i in indexes:
            allel_1 = allels[i].value_counts().rename_axis('Nb').reset_index(name='counts')
            allel_2 = allels[i+1].value_counts().rename_axis('Nb').reset_index(name='counts')

            if len(allel_1['counts']) == 2 and len(allel_2['counts']) == 2: 
                alt_nb = allel_1['Nb'][1]
                alt_c = allel_1['counts'][1]+allel_2['counts'][1]
            elif len(allel_1['counts']) == 2:
                alt_nb = allel_1['Nb'][1]
                alt_c = allel_1['counts'][1]
            else: 
                alt_nb = allel_2['Nb'][1]
                alt_c = allel_2['counts'][1]

            info = {'ref_Nb':allel_1['Nb'][0],
                'c_ref':(allel_1['counts'][0]+allel_2['counts'][0]),
                'alt_Nb':allel_1['Nb'][1] if len(allel_1['Nb']) == 2 else allel_2['Nb'][1],
                'c_alt':alt_c}
            freq_count.append(info)

In [3]:
#I could not try this one, but should work properly
def whole_main():
    allels = pd.read_csv('filtered_conv.ped',sep = '\t',header=None)
    indexes = np.arange(6,248190,2)
    for i in indexes:
        allel_1 = allels[i].value_counts().rename_axis('Nb').reset_index(name='counts')
        allel_2 = allels[i+1].value_counts().rename_axis('Nb').reset_index(name='counts')

        if len(allel_1['counts']) == 2 and len(allel_2['counts']) == 2: 
            alt_nb = allel_1['Nb'][1]
            alt_c = allel_1['counts'][1]+allel_2['counts'][1]
        elif len(allel_1['counts']) == 2:
            alt_nb = allel_1['Nb'][1]
            alt_c = allel_1['counts'][1]
        else: 
            alt_nb = allel_2['Nb'][1]
            alt_c = allel_2['counts'][1]

        info = {'ref_Nb':allel_1['Nb'][0],
                'c_ref':(allel_1['counts'][0]+allel_2['counts'][0]),
                'alt_Nb':allel_1['Nb'][1] if len(allel_1['Nb']) == 2 else allel_2['Nb'][1],
                 'c_alt':alt_c}
        freq_count.append(info)

In [4]:
#Takes about 12 minutes to run but does not require too much memory; if not needed, uncomment whole_main()
freq_count = []
main()
#whole_main()

In [5]:
#Manipulate the generated dictionary and exports it as allele_freq.txt
df = pd.DataFrame(freq_count)
snp_id = pd.read_csv('filtered.bim',sep = '\t',header=None, dtype={0:int, 1:str, 2:int, 3:int, 4:str, 5:str}, usecols=[1]).rename(columns = {1:'ID'})
snp_id['ID'] = 'Chr'+snp_id['ID'].replace('-','/', regex = True)
df_export = pd.DataFrame({'ID':snp_id['ID'],'Ref':df['ref_Nb'],'Alt':df['alt_Nb'],'Freq':df['c_alt']/214})
df_export.to_csv(r'allele_freq.txt', header=True, index=None, sep='\t', mode='w')